In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastrl['dev']  # upgrade fastrl on colab

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

In [ ]:
# default_exp data.block

In [ ]:
# export
# Python native modules
import os
from collections import deque
from time import sleep
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from torch.utils.data import Dataset
from torch import nn
import torch

# Local modules
from fastrl.core import *

In [ ]:
import numpy as np
import gym
import time,sys
import torch.multiprocessing as mp
import pandas as pd


# Data Block
> Fastrl transforms for iterating through environments

In [ ]:
# export
class DQN(Module):
    def __init__(self):
        self.policy=nn.Sequential(
            nn.Linear(4,50),
            nn.ReLU(),
            nn.Linear(50,2),
            nn.ReLU()
        )
    
    def forward(self,x): 
        return torch.argmax(self.policy(x),dim=0)

Development of this was helped by [IterableData documentation on multiple workers](https://github.com/pytorch/pytorch/blob/4949eea0ffb60dc81a0a78402fa59fdf68206718/torch/utils/data/dataset.py#L64)

This code is heavily modifed from https://github.com/Shmuma/ptan

Reference for env [semantics related to vectorized environments](https://github.com/openai/universe/blob/master/doc/env_semantics.rst)

Useful links:
- [torch multiprocessing](https://github.com/pytorch/pytorch/blob/a61a8d059efa0fb139a09e479b1a2c8dd1cf1a44/torch/utils/data/dataloader.py#L564)
- [torch worker](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/_utils/worker.py)

In [ ]:
D.mapv?

Signature: D.mapv(self, f, *args, gen=False, wise='value', **kwargs)
Docstring: <no docstring>
File:      ~/fastrl/fastrl/core.py
Type:      function


In [ ]:
# export
def _listify_if_single(o,single): 
    if not isinstance(o,(np.ndarray,Tensor)) and single:
        return L(o)
    if isinstance(o,(np.ndarray)) and single:
        return np.expand_dims(o,0) if len(o.shape)==1 else o
    if isinstance(o,(np.ndarray)) and single:
        return o.unsqueeze(0) if len(o.size())==1 else o
    return o

def init_experience(single=False,but='',**kwargs): 
    "Returns dictionary with default values that can be overridden."
    experience=D(
        state=0,action=0,next_state=0,reward=0,done=False,
        step=0,steps=0,n_env=0,image=0
    )
    for s in but.split(','):
        if s in experience:del experience[s]
    return D(merge(experience,kwargs)).mapv(_listify_if_single,single=single)

In [ ]:
init_experience()

{'state': 0,
 'action': 0,
 'next_state': 0,
 'reward': 0,
 'done': False,
 'step': 0,
 'steps': 0,
 'n_env': 0,
 'image': 0}

In [ ]:
init_experience(but='image,step')

{'state': 0,
 'action': 0,
 'next_state': 0,
 'reward': 0,
 'done': False,
 'steps': 0,
 'n_env': 0}

In [ ]:
sum([init_experience(),init_experience()],init_experience())

{'state': (#3) [0,0,0],
 'action': (#3) [0,0,0],
 'next_state': (#3) [0,0,0],
 'reward': (#3) [0,0,0],
 'done': (#3) [False,False,False],
 'step': (#3) [0,0,0],
 'steps': (#3) [0,0,0],
 'n_env': (#3) [0,0,0],
 'image': (#3) [0,0,0]}

In [ ]:
# export
def _state2experience(s,**kwargs):   return init_experience(state=s,**kwargs)
def _env_reset(o):                   return o.reset()
def _env_seed(o,seed):               return o.seed(seed)
def _env_render(o,mode='rgb_array'): return [o.render(mode=mode).copy()]
def _env_step(o,*args,**kwargs):     return o.step(*args,**kwargs)

class FakeAgent:
    def __init__(self,action_space): store_attr()
    def __call__(self,state,**kwargs):
        return L([self.action_space.sample() for _ in range(state.shape[0])]),D(kwargs)

class ExperienceSource(Stateful):
    _stateattrs=('pool',)
    def __init__(self,env:str,agent=None,n_envs:int=1,steps_count:int=1,steps_delta:int=1,
                 seed:int=None,render=None,num_workers=0,but='',**kwargs):
        store_attr()
        self.env_kwargs=kwargs
        self.pool=L()
        if self.render is None: self.but+=',image'

    def _init_state(self):
        "Inits the histories, experiences, and the environment pool when sent to a `Process`"
        self.history,self.pool=L((deque(maxlen=self.steps_count),
                                  gym.make(self.env,**self.env_kwargs)) 
                                  for _ in range(self.n_envs)).zip().map(L) 
        self.pool.map(_env_seed,seed=self.seed)
        if self.agent is None: self.agent=FakeAgent(self.pool[0].action_space)
        self.reset_all()
        
    def reset_all(self):
        self.experiences=self.pool.map(_env_reset)
        self.experiences=self.experiences.map(_state2experience,but=self.but,single=self.n_envs==1)
        if self.n_envs>1:self.experiences=sum(self.experiences[1:],self.experiences[0])
        else:            self.experiences=self.experiences[0]
        self.attempt_render(self.experiences)
        
    def attempt_render(self,experiences,indexes=None):
        if self.render is not None: 
            pool=self.pool if indexes is None else self.pool[indexes]
            renders=pool.map(_env_render,mode=self.render)
            # No idea why we have to do this, but multiprocessing hangs forever otherwise
            if self.num_workers>0:sleep(0.1) 
            experiences['image']=np.vstack(renders).astype(float)

    def __iter__(self):
        "Iterates through a list of environments."
        if not self.pool:self._init_state()
        while True: 
#             try:
#             print(self.experiences)
            not_done_idxs=self.experiences.argwhere('done',L.argwhere,lambda x:x==False)
            not_done_experiences=self.experiences.filter(indexes=not_done_idxs)
#             print(not_done_experiences)
            actions,experiences=self.agent(**not_done_experiences)
            step_res=self.pool[not_done_idxs].zipwith(actions).starmap(_env_step)
            next_states,(rewards,dones,info)=step_res.zip()[0],step_res.zip()[1:].map(L)
            
            self.attempt_render(self.experiences,not_done_idxs)

            experiences=D(merge(not_done_experiences,experiences,
                                D(next_state=next_states,reward=rewards,done=dones)))
            if self.n_envs>1:
                split_experiences=parallel(partial(experiences.subset),not_done_idxs,
                                           threadpool=True,n_workers=2,progress=False)
                yield split_experiences
            else:
                print(experiences)
                yield experiences # {'actions':actions}
#             except ValueError:
#                 self.reset_all()
            
add_docs(ExperienceSource,
        """Iterates through `n_envs` of `env` feeding experience or states into `agent`.
           If `agent` is None, then random actions will be taken instead.
           It will return `steps_count` experiences every `steps_delta`.
           At the end of an env, it will return `steps_count-1` experiences per next. """,
        reset_all="resets the envs and experience",
        attempt_render="Updates `experiences` with images if `render is not None`. Optionally indexes can be passed.")

In [ ]:
# export
class SourceDataset(IterableDataset):
    "Iterates through a `source` object. Allows for re-initing source connections when `num_workers>0`"
    def __init__(self,source=None): store_attr('source')
    def __iter__(self):             return iter(self.source)
    def wif(self):                  self.source._init_state()

In [ ]:
source=ExperienceSource('CartPole-v1',None,n_envs=2)
dataset=SourceDataset(source)

In [ ]:
data=None
for x in DataLoader(dataset,num_workers=0,n=10,persistent_workers=True,wif=dataset.wif):
    data=D(x) if data is None else data+D(x)
data.pandas()

,state,action,next_state,reward,done,step,steps,n_env,state_mu,next_state_mu
0,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,-0.006892,-0.034632
1,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,0.009369,-0.013647
2,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,-0.006892,-0.014316
3,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,0.009369,0.011686
4,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,-0.006892,-0.042595
5,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,0.009369,-0.011611
6,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,-0.006892,-0.022921
7,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,0.009369,-0.035306
8,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,-0.006892,-0.003363
9,"torch.Size([20, 4])",0,"torch.Size([20, 4])",1.0,False,0,0,0,0.009369,-0.011064


`ExperienceSource` is designed for iterating through `n_envs` environments.

A single experience is a `dict`:

In [ ]:
D(state=None,action=None,next_state=None,reward=None,rewards=None,
             step=None,steps=None)

However, an agent has full power to add fields to this dict wile running

In [ ]:
show_doc(ExperienceSource._init_state)

In [ ]:
show_doc(ExperienceSource.__iter__)

If the `self.pool` field is empty, it will call `_init_state` to reinitialize everything.

In [ ]:

import gym
from queue import deque

class ExperienceSource(object):
    def __init__(self, env:str,agent,n_envs:int=1,steps_count:int=1,steps_delta:int=1,
                 vectorized:bool=False,seed:int=None):
        store_attr()
        self.pool:List[gym.Env]=[gym.make(self.env) for _ in range(self.n_envs)]
    
    def init_env(self,env,states,histories,cur_rewards,cur_steps,env_lens):
        env.seed(self.seed)
        obs=env.reset()
        if self.vectorized:
            obs_len = len(obs)
            states.extend(obs)
        else:
            obs_len = 1
            states.append(obs)
        env_lens.append(obs_len)

        for _ in range(obs_len):
            histories.append(deque(maxlen=self.steps_count))
            cur_rewards.append(0.0)
            cur_steps.append(0)
        
    def __iter__(self):
        states,histories,cur_rewards,cur_steps,env_lens=[],[],[],[],[]

        for env in self.pool: self.init_env(env,states,histories,cur_rewards,cur_steps,env_lens)

        iter_idx = 0
        while True:
            actions = [None] * len(states)
            states_input = []
            states_indices = []
            for idx, state in enumerate(states):
                if state is None:
                    actions[idx] = self.pool[0].action_space.sample()  # assume that all envs are from the same family
                else:
                    states_input.append(state)
                    states_indices.append(idx)
            if states_input:
                states_actions = self.agent(states_input)
                for idx, action in enumerate(states_actions):
                    g_idx = states_indices[idx]
                    actions[g_idx] = action
#             grouped_actions = _group_list(actions, env_lens)
            grouped_actions=np.split(actions,env_lens[:])

            global_ofs = 0
            for env_idx, (env, action_n) in enumerate(zip(self.pool, grouped_actions)):
                if self.vectorized:
                    next_state_n, r_n, is_done_n, _ = env.step(action_n)
                else:
                    next_state, r, is_done, _ = env.step(action_n[0])
                    next_state_n, r_n, is_done_n = [next_state], [r], [is_done]

                for ofs, (action, next_state, r, is_done) in enumerate(zip(action_n, next_state_n, r_n, is_done_n)):
                    idx = global_ofs + ofs
                    state = states[idx]
                    history = histories[idx]

                    cur_rewards[idx] += r
                    cur_steps[idx] += 1
                    if state is not None:
                        history.append(dict(state=state,next_state=next_state, action=action, reward=r, done=is_done,steps=cur_steps[idx],episode_reward=cur_rewards[idx],env=env_idx))
                    if len(history) == self.steps_count and iter_idx % self.steps_delta == 0:
                        yield tuple(history)
                    states[idx] = next_state
                    if is_done:
                        # in case of very short episode (shorter than our steps count), send gathered history
                        if 0 < len(history) < self.steps_count:
                            yield tuple(history)
                        # generate tail of history
                        while len(history) > 1:
                            history.popleft()
                            yield tuple(history)
                        cur_rewards[idx] = 0.0
                        cur_steps[idx] = 0
                        # vectorized envs are reset automatically
                        env.seed(self.seed)
                        states[idx]=env.reset() if not self.vectorized else None
                        history.clear()
                global_ofs += len(action_n)
            iter_idx += 1

In [ ]:
agent=lambda s: [0]

data=None

for e,_ in zip(ExperienceSource('CartPole-v1',agent,seed=0,steps_count=3),range(70)):
    data=D(e) if data is None else data+D(e)
data.pandas()

In [ ]:
# export
class TestDataset(IterableDataset):
    def __init__(self,start=1,end=10,policy=None,device='cpu',n_envs=1):
        store_attr('start,end,policy,device,n_envs')
        
    def init_envs(self,n):
        self.envs=[gym.make('CartPole-v1') for i in range(n)]
        
    def __iter__(self):
        worker_info=torch.utils.data.get_worker_info()
        
        if worker_info is None:  # single-process data loading, return the full iterator
            self.init_envs(self.n_envs)
        else:  # in a worker process
            # split workload
            per_worker=int(math.ceil(self.n_envs/worker_info.num_workers))
            self.init_envs(per_worker)
        return iter(range(iter_start, iter_end))

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():   
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()